In [ ]:
#%pip install requests
#%pip install telethon
#%pip install --upgrade ipykernel
#%pip install requests lxml pandas
#%pip install selenium
#%pip install pymongo
#%pip install requests lxml mysql-connector-python
#%pip install requests beautifulsoup4 lxml pymongo

Задание: Парсинг и добавление новых записей в MongoDB

In [ ]:
#Импортируем необходимые библиотеки
from bs4 import BeautifulSoup
import requests
from pymongo import MongoClient, errors
from collections import OrderedDict  # для гарантии порядка полей

#Интернет-страница для парсинга таблицы
CATALOG_URL = "https://www.old-games.ru/catalog/?platform=1&sort=popularity&ysclid=mgtig0uzj8331895606"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
}

#Данные нашей локальной базы данных в MongoDB
MONGO_URI = "mongodb://localhost:27017"
DB_NAME = "oldgames"
COLL_NAME = "games"

#Функция для получения числа из строки
def first_number(value):
    if value is None:
        return None
    s = str(value)
    num, started = "", False
    for ch in s:
        if ch.isdigit():
            num += ch
            started = True
        elif started:
            break
    return int(num) if num else None

#Функция для парсинга интернет-страницы
def parse_catalog(url: str):
    
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    r.encoding = r.apparent_encoding

    soup = BeautifulSoup(r.text, "lxml")
    table = soup.select_one("table.listtable")
    if not table:
        raise RuntimeError("Не найдена таблица 'listtable'.")

    docs = []
    for tr in table.select('tr[id^="game_"]'):
        rid = tr.get("id", "").strip()  # "game_604"
        if not rid.startswith("game_"):
            continue
        try:
            site_id = int(rid.replace("game_", "", 1))
        except ValueError:
            continue

        tds = tr.find_all("td", recursive=False)
        if len(tds) < 6:
            continue

        a = tds[0].find("a", href=True)
        title = (a.get_text(strip=True) if a else tds[0].get_text(strip=True)) or None

        genre_links = [x.get_text(strip=True) for x in tds[1].find_all("a")]
        genre = (", ".join(genre_links) if genre_links else tds[1].get_text(strip=True)) or None

        year = first_number(tds[2].get_text(strip=True))
        platform = (tds[3].get_text(strip=True) or None)
        publisher = (tds[4].get_text(strip=True) or None)

        rating_img = tds[5].find("img")
        rating_text = rating_img.get("alt", "") if rating_img else ""
        rating = first_number(rating_text)

        # Собираем документ 
        doc = OrderedDict()
        doc["title"] = title
        doc["genre"] = genre
        doc["year"] = year
        doc["platform"] = platform
        doc["publisher"] = publisher
        doc["rating"] = rating
        doc["id"] = site_id  # числовой id игры

        docs.append(doc)
    return docs

#Функция для обновления записей в MongoDB по уникальному id игры
def insert_only_new_by_id(docs):
    
    #Получаем доступ к базе данных MongoDB
    client = MongoClient(MONGO_URI)
    col = client[DB_NAME][COLL_NAME]

    # Задаем уникальный индекс по id игры, чтобы избежать дублей
    try:
        col.create_index("id", unique=True, name="uniq_id")
    except errors.PyMongoError:
        pass  

    #Создаем список уникальных id в нашей базе данных
    existing_ids = set(col.distinct("id"))

    #Оставляем только те строки таблицы, id которых еще в БД не встречались
    new_docs = [d for d in docs if d.get("id") is not None and d["id"] not in existing_ids]

    #Добавляем в mongoDB новые записи
    added = []
    for d in new_docs:
        try:
            res = col.insert_one(d)
            show = OrderedDict([("_id", res.inserted_id)])
            show.update(d)
            added.append(show)
        except errors.DuplicateKeyError:
            pass

    return added

#Безопасно запсукаем эти функции
try:
    parsed = parse_catalog(CATALOG_URL)
    added = insert_only_new_by_id(parsed)
    
    #Выводим все добавленные записи в БД
    print(f"Добавлено новых записей: {len(added)}")
    if added:
        # Печатаем записи в нужном порядке: _id, title, genre, year, platform, publisher, rating, id
        for i, d in enumerate(added, 1):
            print(
                f"{i}. _id={d['_id']}, title={d['title']!r}, genre={d['genre']!r}, "
                f"year={d['year']}, platform={d['platform']!r}, publisher={d['publisher']!r}, "
                f"rating={d['rating']}, id={d['id']}"
            )
    else:
        print("Новых записей не найдено — все id уже есть в базе.")
except Exception as e:
    print("Ошибка:", e)


Добавлено новых записей: 1
1. _id=690dd40ca53cc6d0633e7246, title='Blood', genre='Action', year=1997, platform='DOS', publisher='GT Interactive Software', rating=10, id=11


Задание: Выполение поиска по базе данных в MongoDB

In [ ]:

from pymongo import MongoClient

#Данные нашей локальной базы данных в MongoDB
MONGO_URI = "mongodb://localhost:27017"
DB_NAME    = "oldgames"
COLL_NAME  = "games"

# порядок полей для вывода
DISPLAY_ORDER = ["_id", "title", "genre", "year", "platform", "publisher", "rating", "id"]

# меню выбора: номер -> (поле, тип)
MENU = {
    1: ("title",     "str"),
    2: ("genre",     "str"),
    3: ("year",      "int"),
    4: ("platform",  "str"),
    5: ("publisher", "str"),
    6: ("rating",    "int"),
    7: ("id",        "int"),
}

#Функция для получения корректного 24-символьного id записи в MongoDB
def _id_hex(value):
    try:
        return value.binary.hex() 
    except Exception:
        return str(value)
    
#Функция для интерактивного поиска
def interactive_search(limit: int = 500):
    client = MongoClient(MONGO_URI)
    col = client[DB_NAME][COLL_NAME]

    # меню
    print("Выберите поле для поиска (введите номер):")
    for k in sorted(MENU):
        print(f" {k}. {MENU[k][0]}")
    try:
        choice = int(input("Номер поля: ").strip())
    except Exception:
        print("Нужно ввести число из меню.")
        return

    if choice not in MENU:
        print("Нет такого пункта меню.")
        return

    field, ftype = MENU[choice]
    print(f"отлично, вы выбрали параметр для поиска {field}, теперь введите его значение:")
    raw = input().strip()
    if raw == "":
        print("Пустое значение — поиск отменён.")
        return

    # фильтр
    if ftype == "str":
        flt = {field: {"$regex": raw, "$options": "i"}}  # подстрока, без учёта регистра
    else:
        try:
            flt = {field: int(raw)}
        except ValueError:
            print(f"Поле '{field}' числовое. Введите целое число.")
            return

    docs = list(col.find(flt).limit(limit))
    print(f"\nНайдено документов: {len(docs)} (limit={limit})")
    if not docs:
        return

    # печать в точном формате
    for i, d in enumerate(docs, 1):
        _id_str = _id_hex(d.get("_id"))
        title   = d.get("title")
        genre   = d.get("genre")
        year    = d.get("year")
        platform= d.get("platform")
        publisher=d.get("publisher")
        rating  = d.get("rating")
        game_id = d.get("id")

        line = (f"{i}. _id={_id_str}, "
                f"title={repr(title)}, "
                f"genre={repr(genre)}, "
                f"year={year}, "
                f"platform={repr(platform)}, "
                f"publisher={repr(publisher)}, "
                f"rating={rating}, "
                f"id={game_id}")
        print(line)

# запускаем
interactive_search()


Выберите поле для поиска (введите номер):
 1. title
 2. genre
 3. year
 4. platform
 5. publisher
 6. rating
 7. id
отлично, вы выбрали параметр для поиска rating, теперь введите его значение:

Найдено документов: 2 (limit=500)
1. _id=690dbd2e30f98410db9c0c56, title='Dangerous Dave in the Haunted Mansion', genre='Arcade', year=1991, platform='DOS', publisher='Softdisk Publishing', rating=6, id=159
2. _id=690dbd2e30f98410db9c0c71, title='I Have No Mouth, and I Must Scream', genre='Quest', year=1995, platform='DOS', publisher='Cyberdreams', rating=6, id=153
